# Tutorial:
## Logistic regression for factor mapping
This analysis performs scenario discovery through the use of logistic regression
on three water users in a multi-stakeholder basin.

The analysis is performed on data generated during a previous experiment, detailed at
[Hadjimichael et al. (2020)](https://agupubs.onlinelibrary.wiley.com/doi/abs/10.1029/2020EF001503).

The experiment employed exploratory modeling to assess the impacts of several (hydrologic,
infrastructural, societal) uncertain factors on water shortages experienced in a river basin in
Colorado.

<img src="./figs/basin_map.pdf" width="300">

Focusing on decision-relevant metrics, the scenario discovery is applied to the shortages experienced
by each individual user (i.e., not on a single basin-wide or sector-wide metric). For this training
example, we'll be using three different water users, two irrigation users and one municipal user.

In [1]:
import pandas as pd
from utils import *
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

## Step 1:  Load Latin Hypercube Sample and set up problem

In [ ]:
all_IDs = ['7000550','7200799','3704614']
nStructures = len(all_IDs)
LHsamples = np.loadtxt('./data/LHsamples_original_1000.txt')
param_bounds=np.loadtxt('./data/uncertain_params_bounds.txt', usecols=(1,2))
realizations = 10

param_names=['IWRmultiplier','RESloss','TBDmultiplier','M_Imultiplier',
             'Shoshone','ENVflows','EVAdelta','XBM_mu0','XBM_sigma0',
             'XBM_mu1','XBM_sigma1','XBM_p00','XBM_p11', 'shift']

SOW_values = np.array([1,1,1,1,0,0,1,1,1,1,1,0,0,0]) #Default parameter values for base SOW

# Set arrays for shortage frequencies and magnitudes
frequencies = np.arange(10, 110, 10)
magnitudes = np.arange(10, 110, 10)

heatmaps = [load_performance(all_IDs[i])/100 for i in range(len(all_IDs))]
scores = [pd.read_csv(all_IDs[i] + '_pseudo_r_scores.csv', sep=",") for i in range(len(all_IDs))]

freq = [1,0,0]
mag = [7,3,7]

fig, axes = plt.subplots(1,3, dpi=600)
for i in range(len(axes.flat)):
    ax = axes.flat[i]
    allSOWsperformance = heatmaps[i]
    all_pseudo_r_scores = scores[i]
    dta = pd.DataFrame(data = np.repeat(LHsamples, realizations, axis = 0), columns=param_names)
    dta['Success']=allSOWsperformance[freq[i],mag[i],:]
    pseudo_r_scores=all_pseudo_r_scores[str(frequencies[freq[i]])+'yrs_'+str(magnitudes[mag[i]])+'prc'].values
    top_predictors = np.argsort(pseudo_r_scores)[::-1][:2] #Sort scores and pick top 2 predictors
    # define color map for dots representing SOWs in which the policy
    # succeeds (light blue) and fails (dark red)
    dot_cmap = mpl.colors.ListedColormap(np.array([[227,26,28],[166,206,227]])/255.0)
    # define color map for probability contours
    contour_cmap = mpl.cm.get_cmap('RdBu')
    # define probability contours
    contour_levels = np.arange(0.0, 1.05,0.1)
    # define base values of the predictors
    base = SOW_values[top_predictors]
    ranges = param_bounds[top_predictors]
    # define grid of x (1st predictor), and y (2nd predictor) dimensions
    # to plot contour map over
    xgrid = np.arange(param_bounds[top_predictors[0]][0],
                      param_bounds[top_predictors[0]][1], np.around((ranges[0][1]-ranges[0][0])/500,decimals=4))
    ygrid = np.arange(param_bounds[top_predictors[1]][0],
                      param_bounds[top_predictors[1]][1], np.around((ranges[1][1]-ranges[1][0])/500,decimals=4))
    all_predictors = [ dta.columns.tolist()[i] for i in top_predictors]
    dta['Interaction'] = dta[all_predictors[0]]*dta[all_predictors[1]]
    result = fitLogit(dta, [all_predictors[i] for i in [0,1]])
    contourset = plotContourMap(ax, result, dta, contour_cmap,
                                dot_cmap, contour_levels, xgrid,
                                ygrid, all_predictors[0], all_predictors[1], base)